In [2]:
import numpy as np
import pandas as pd

In [29]:
corr_mat = np.array([[1.0, 0.6], [0.6, 1]])
cov_mat = np.cov(corr_mat, bias=True)
cov_mat

array([[ 0.04, -0.04],
       [-0.04,  0.04]])

In [19]:
n_var = 10
compsub = np.empty([n_var, n_var-1])
for i in range (0, n_var):
    temp = np.arange(n_var)
    compsub[i] = np.delete(temp, i)
    
compsub

array([[1., 2., 3., 4., 5., 6., 7., 8., 9.],
       [0., 2., 3., 4., 5., 6., 7., 8., 9.],
       [0., 1., 3., 4., 5., 6., 7., 8., 9.],
       [0., 1., 2., 4., 5., 6., 7., 8., 9.],
       [0., 1., 2., 3., 5., 6., 7., 8., 9.],
       [0., 1., 2., 3., 4., 6., 7., 8., 9.],
       [0., 1., 2., 3., 4., 5., 7., 8., 9.],
       [0., 1., 2., 3., 4., 5., 6., 8., 9.],
       [0., 1., 2., 3., 4., 5., 6., 7., 9.],
       [0., 1., 2., 3., 4., 5., 6., 7., 8.]])

array([1., 2., 3., 4., 5., 6., 7., 8., 9.])